![image.png](https://i.imgur.com/a3uAqnb.png)

# Contents

In this notebook, we will create a RealNVP(real-valued non-volume preserving) generative model for the Oxford Pets dataset, with an intermediate AutoEncoder.

Instead of training the model on direct pixel values and generating images, we will

1. Train an AutoEncoder for the images
2. Convert the data into embeddings using the AutoEncoder
3. Train our RealNVP model on the embeddings
4. Generate embeddings using the RealNVP model and convert them to images using the AutoEncoder's decoder

This notebook is heavily based on [This Repo](https://github.com/SpencerSzabados/realnvp-pytorch/tree/master)

In [ ]:
!pip install -U datasets

In [1]:
import copy
import os

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torch import nn, distributions
from torch.nn import MSELoss
from torchvision import transforms
from datasets import load_dataset


In [2]:
class LinearBatchNorm(nn.Module):
    """
    An (invertible) batch normalization layer.
    This class is mostly inspired from this one:
    https://github.com/kamenbliznashki/normalizing_flows/blob/master/maf.py
    """

    def __init__(self, input_size, momentum=0.9, eps=1e-5):
        super().__init__()
        self.momentum = momentum
        self.eps = eps

        self.log_gamma = nn.Parameter(torch.zeros(input_size))
        self.beta = nn.Parameter(torch.zeros(input_size))

        self.register_buffer('running_mean', torch.zeros(input_size))
        self.register_buffer('running_var', torch.ones(input_size))

    def forward(self, x, **kwargs):
        if self.training:
            self.batch_mean = x.mean(0)
            self.batch_var = x.var(0)

            self.running_mean.mul_(self.momentum).add_(self.batch_mean.data * (1 - self.momentum))
            self.running_var.mul_(self.momentum).add_(self.batch_var.data * (1 - self.momentum))

            mean = self.batch_mean
            var = self.batch_var
        else:
            mean = self.running_mean
            var = self.running_var

        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        y = self.log_gamma.exp() * x_hat + self.beta

        log_det = self.log_gamma - 0.5 * torch.log(var + self.eps)

        return y, log_det.expand_as(x).sum(1)

    def backward(self, x, **kwargs):
        if self.training:
            mean = self.batch_mean
            var = self.batch_var
        else:
            mean = self.running_mean
            var = self.running_var

        x_hat = (x - self.beta) * torch.exp(-self.log_gamma)
        x = x_hat * torch.sqrt(var + self.eps) + mean

        log_det = 0.5 * torch.log(var + self.eps) - self.log_gamma

        return x, log_det.expand_as(x).sum(1)


class LinearCouplingLayer(nn.Module):
    """
    Linear coupling layer.
        (i) Split the input x into 2 parts x1 and x2 according to a given mask.
        (ii) Compute s(x2) and t(x2) with given neural network.
        (iii) Final output is [exp(s(x2))*x1 + t(x2); x2].
    The inverse is trivially [(x1 - t(x2))*exp(-s(x2)); x2].
    """

    def __init__(self, input_dim, mask, network_topology, conditioning_size=None, single_function=True):
        super().__init__()

        if conditioning_size is None:
            conditioning_size = 0

        if network_topology is None or len(network_topology) == 0:
            network_topology = [input_dim]

        self.register_buffer('mask', mask)

        self.dim = input_dim

        self.s = [nn.Linear(input_dim + conditioning_size, network_topology[0]), nn.ReLU()]

        for i in range(len(network_topology)):
            t = network_topology[i]
            t_p = network_topology[i - 1]
            self.s.extend([nn.Linear(t_p, t), nn.ReLU()])

        if single_function:
            input_dim = input_dim * 2

        ll = nn.Linear(network_topology[-1], input_dim)

        self.s.append(ll)
        self.s = nn.Sequential(*self.s)

        if single_function:
            self.st = lambda x: (self.s(x).chunk(2, 1))
        else:
            self.t = copy.deepcopy(self.s)
            self.st = lambda x: (self.s(x), self.t(x))

    def backward(self, x, y=None):
        mx = x * self.mask

        if y is not None:
            _mx = torch.cat([y, mx], dim=1)
        else:
            _mx = mx

        s, t = self.st(_mx)
        s = torch.tanh(s)

        u = mx + (1 - self.mask) * (x - t) * torch.exp(-s)

        log_abs_det_jacobian = - (1 - self.mask) * s

        return u, log_abs_det_jacobian.sum(1)

    def forward(self, u, y=None):
        mu = u * self.mask

        if y is not None:
            _mu = torch.cat([y, mu], dim=1)
        else:
            _mu = mu

        s, t = self.st(_mu)
        s = torch.tanh(s)

        x = mu + (1 - self.mask) * (u * s.exp() + t)

        log_abs_det_jacobian = (1 - self.mask) * s

        return x, log_abs_det_jacobian.sum(1)


class Permutation(nn.Module):
    """
    A permutation layer.
    """

    def __init__(self, in_ch):
        super().__init__()
        self.in_ch = in_ch
        self.register_buffer('p', torch.randperm(in_ch))
        self.register_buffer('invp', torch.argsort(self.p))

    def forward(self, x, y=None):
        assert x.shape[1] == self.in_ch
        out = x[:, self.p]
        return out, 0

    def backward(self, x, y=None):
        assert x.shape[1] == self.in_ch
        out = x[:, self.invp]
        return out, 0


class SequentialFlow(nn.Sequential):
    """
    Utility class to build a normalizing flow from a sequence of base transformations.
    During forward and inverse steps, aggregates the sum of the log determinants of the Jacobians.
    """

    def forward(self, x, y=None):
        log_det = 0
        for module in self:
            x, _log_det = module(x, y=y)
            log_det = log_det + _log_det
        return x, log_det

    def backward(self, u, y=None):
        log_det = 0
        for module in reversed(self):
            u, _log_det = module.backward(u, y=y)
            log_det = log_det + _log_det
        return u, log_det

    def forward_steps(self, x, y=None):
        log_det = 0
        xs = [x]
        for module in self:
            x, _log_det = module(x, y=y)
            xs.append(x)
            log_det = log_det + _log_det
        return xs, log_det

    def backward_steps(self, u, y=None):
        log_det = 0
        us = [u]
        for module in reversed(self):
            u, _log_det = module.backward(u, y=y)
            us.append(u)
            log_det = log_det + _log_det
        return us, log_det


class LinearRNVP(nn.Module):
    """
    Main RNVP model, alternating affine coupling layers
    with permutations and/or batch normalization steps.
    """

    def __init__(self, input_dim, coupling_topology, flow_n=2, use_permutation=False,
                 batch_norm=False, mask_type='odds', conditioning_size=None, single_function=False):
        super().__init__()

        self.register_buffer('prior_mean', torch.zeros(input_dim))
        self.register_buffer('prior_var', torch.ones(input_dim))

        if mask_type == 'odds':
            mask = torch.arange(0, input_dim).float() % 2
        elif mask_type == 'half':
            mask = torch.zeros(input_dim)
            mask[:input_dim // 2] = 1
        else:
            assert False

        if coupling_topology is None:
            coupling_topology = [input_dim // 2, input_dim // 2]

        blocks = []

        for i in range(flow_n):

            blocks.append(LinearCouplingLayer(input_dim, mask, network_topology=coupling_topology,
                                              conditioning_size=conditioning_size, single_function=single_function))

            if use_permutation:
                blocks.append(Permutation(input_dim))
            else:
                mask = 1 - mask

            if batch_norm:
                blocks.append(LinearBatchNorm(input_dim))

        self.flows = SequentialFlow(*blocks)

    def logprob(self, x):
        return self.prior.log_prob(x)

    @property
    def prior(self):
        return distributions.Normal(self.prior_mean, self.prior_var)

    def forward(self, x, y=None, return_step=False):
        if return_step:
            return self.flows.forward_steps(x, y)
        return self.flows.forward(x, y)

    def backward(self, u, y=None, return_step=False):
        if return_step:
            return self.flows.backward_steps(u, y)
        return self.flows.backward(u, y)

    def sample(self, samples=1, y=None, return_step=False, return_logdet=False):
        u = self.prior.sample((samples,))
        z, d = self.backward(u, y=y, return_step=return_step)
        if return_logdet:
            d = self.logprob(u).sum(1) + d
            return z, d
        return z

In [3]:
# Set the random seeds
torch.manual_seed(0)
np.random.seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Loading Data

In [ ]:
# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the Oxford Pets dataset from Hugging Face
dataset = load_dataset("enterprise-explorers/oxford-pets")
train_dataset = dataset['train']


# Create a custom dataset class for the Oxford Pets data
class OxfordPetsDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']

        # Convert to RGB if not already
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        # Use the 'dog' field: True for dog (1), False for cat (0)
        label = 1 if item['dog'] else 0

        return image, label


train_set = OxfordPetsDataset(train_dataset, transform=transform)

BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(train_set, BATCH_SIZE, shuffle=True)

### Autoencoder definition

# **📌 Autoencoder for Compressing Pet Images**

The Oxford Pets dataset contains much more complex images (64x64, RGB, varied poses and backgrounds) than MNIST. To handle this, we use a more powerful **Autoencoder** with a deeper, DCGAN-style architecture.

## **🔹 How it Works**

1️⃣ **Encoder**: A deep convolutional network that compresses a 64x64 pet image into a low-dimensional latent vector (embedding). This embedding must capture the key features that define the pet's appearance and species.

2️⃣ **Decoder**: A deconvolutional network that attempts to perfectly reconstruct the original image from this compressed embedding.

By training this model to minimize reconstruction error, we create a rich, low-dimensional "embedding space" that we can then model with our Normalizing Flow.



## **📌 Expected Input & Output Shapes**

- **Input Image:** `(batch_size, 3, 64, 64)`
- **Latent Embedding:** `(batch_size, 256)`  *(256 is our `EMBEDDING_DIM`)*
- **Reconstructed Image:** `(batch_size, 3, 64, 64)`

In [ ]:
EMBEDDING_DIM = 256

In [ ]:
class ImprovedAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            # 64x64x3 -> 32x32x64
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            # 32x32x64 -> 16x16x128
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 16x16x128 -> 8x8x256
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            # 8x8x256 -> 4x4x512
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            # 4x4x512 -> 2x2x512
            nn.Conv2d(512, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        # Bottleneck layers
        self.flatten_size = 512 * 2 * 2  # 2048
        self.fc_encode = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.2),
            nn.Linear(1024, EMBEDDING_DIM)
        )

        self.fc_decode = nn.Sequential(
            nn.Linear(EMBEDDING_DIM, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(1024, self.flatten_size),
            nn.BatchNorm1d(self.flatten_size),
            nn.ReLU(inplace=True)
        )

        # Decoder with symmetric architecture
        self.decoder = nn.Sequential(
            # 2x2x512 -> 4x4x512
            nn.ConvTranspose2d(512, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            # 4x4x512 -> 8x8x256
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            # 8x8x256 -> 16x16x128
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            # 16x16x128 -> 32x32x64
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            # 32x32x64 -> 64x64x3
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=True),
            nn.Tanh()
        )

        # Initialize weights properly
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, (nn.Conv2d, nn.ConvTranspose2d)):
            nn.init.normal_(module.weight, 0.0, 0.02)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.BatchNorm2d):
            nn.init.normal_(module.weight, 1.0, 0.02)
            nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, 0.0, 0.02)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def encode(self, x):
        # Encoder forward pass
        features = self.encoder(x)
        features_flat = features.view(features.size(0), -1)
        embedding = self.fc_encode(features_flat)
        return embedding

    def decode(self, embedding):
        # Decoder forward pass
        features_flat = self.fc_decode(embedding)
        features = features_flat.view(-1, 512, 2, 2)
        reconstruction = self.decoder(features)
        return reconstruction

    def forward(self, x):
        embedding = self.encode(x)
        reconstruction = self.decode(embedding)
        return reconstruction, embedding


### Autoencoder training on Pets

In [ ]:
autoencoder = ImprovedAutoEncoder()
autoencoder = autoencoder.to(device)

criterion = MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-4, weight_decay=1e-5)

AE_EPOCHS = 100

In [ ]:
epoch_losses = []
for j in range(AE_EPOCHS):

    losses = []
    for batch_idx, data in enumerate(tqdm(train_loader)):
        x, _ = data
        x = x.to(device)

        # Run the autoencoder
        _x, emb = autoencoder(x)
        # Don't apply sigmoid since we're using tanh output and MSE loss

        # Compute Reconstruction loss - both x and _x are in [-1,1] range
        rec_loss = criterion(_x, x)

        losses.append(rec_loss.item())

        autoencoder.zero_grad()
        rec_loss.backward()
        optimizer.step()
    epoch_losses.append(sum(losses) / len(losses))
    print(f'Epoch #{j + 1}, Loss: {sum(losses) / len(losses):.4f}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(epoch_losses) + 1), epoch_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()


In [ ]:
# Test autoencoder reconstruction quality
autoencoder.eval()

# Get a batch of test images
test_batch_size = 8
test_loader = torch.utils.data.DataLoader(train_set, test_batch_size, shuffle=True)
test_images, test_labels = next(iter(test_loader))
test_images = test_images.to(device)

with torch.no_grad():
    # Get reconstructions
    reconstructed, embeddings = autoencoder(test_images)


# Convert from [-1,1] to [0,1] for display
def denormalize(tensor):
    return (tensor + 1) / 2


# Create visualization
fig, axes = plt.subplots(3, test_batch_size, figsize=(16, 6))

for i in range(test_batch_size):
    # Original image
    orig_img = denormalize(test_images[i]).cpu().permute(1, 2, 0)
    axes[0, i].imshow(torch.clamp(orig_img, 0, 1))
    axes[0, i].set_title('Original')
    axes[0, i].axis('off')

    # Reconstructed image
    recon_img = denormalize(reconstructed[i]).cpu().permute(1, 2, 0)
    axes[1, i].imshow(torch.clamp(recon_img, 0, 1))
    axes[1, i].set_title('Reconstructed')
    axes[1, i].axis('off')

    # Difference (error)
    diff_img = torch.abs(orig_img - recon_img)
    axes[2, i].imshow(diff_img, cmap='hot')
    axes[2, i].set_title('Difference')
    axes[2, i].axis('off')

    species = 'Dog' if test_labels[i] == 1 else 'Cat'
    axes[0, i].text(0.5, -0.1, species, transform=axes[0, i].transAxes,
                    ha='center', fontsize=10, weight='bold')

plt.suptitle('AutoEncoder Reconstruction Quality', fontsize=16)
plt.tight_layout()
plt.show()

### Note: the following is done to make the embeddings in a normalized scale that the NF model expects

In [ ]:
# Recreate embedded dataset with normalization
embedded_data = []

# First pass: collect all embeddings to compute global statistics
all_embeddings = []
autoencoder.eval()

for batch_idx, data in enumerate(tqdm(train_loader, desc="Computing embedding stats")):
    with torch.no_grad():
        x, y = data
        x = x.to(device)
        _, emb = autoencoder(x)
        all_embeddings.append(emb.cpu())

all_embeddings = torch.cat(all_embeddings, dim=0)
emb_mean = all_embeddings.mean(dim=0)
emb_std = all_embeddings.std(dim=0)

print(f"Global embedding mean: {emb_mean.mean():.4f}")
print(f"Global embedding std: {emb_std.mean():.4f}")

# Second pass: create normalized embedded dataset
embedded_data = []
for batch_idx, data in enumerate(tqdm(train_loader, desc="Creating normalized embeddings")):
    with torch.no_grad():
        x, y = data
        x = x.to(device)
        _, emb = autoencoder(x)

        # Normalize embeddings
        emb_normalized = (emb.cpu() - emb_mean) / (emb_std + 1e-8)

        for j in range(len(emb_normalized)):
            embedded_data.append((emb_normalized[j], y[j]))

In [ ]:

embedded_train_loader = torch.utils.data.DataLoader(embedded_data, BATCH_SIZE, shuffle=True)

### Normalizing Flow training

# **📌 RealNVP for Generating Pet Embeddings**
Now that we have a way to represent complex pet images as 256-dimensional vectors, we can train a **Normalizing Flow** to learn the *distribution* of these vectors. We will use the same **RealNVP** model, but this time it will be *conditional*.

## **🔹 Key Concepts**
1️⃣ **Conditional Generation**: We provide the class label (0 for Cat, 1 for Dog) as an additional input to the model. This allows the flow to learn two distinct distributions within the same model and lets us control whether we generate a cat or a dog.

2️⃣ **Invertible Transformation**: The model learns an invertible function `f` that maps a pet embedding `x` and its label `y` to a latent point `z` from a simple Gaussian distribution. This can be reversed to generate a new pet embedding from a random point `z` and a chosen label `y`.

3️⃣ **Coupling Layers**: RealNVP uses these clever layers to split the input, transforming one part based on the other part *and the conditional label*. This makes the transformation powerful while keeping the necessary calculations efficient.



## **📌 Expected Input & Output Shapes**
- **Input (Embeddings):** `(batch_size, 256)`
- **Conditional Input (Labels):** `(batch_size, 2)`  (One-hot encoded: [1,0] for Cat, [0,1] for Dog)
- **Output (Latent `u`):** `(batch_size, 256)`

In [ ]:
FLOW_N = 9  # Number of affine coupling layers
RNVP_TOPOLOGY = [200]  # Size of the hidden layers in each coupling layer

In [ ]:
nf_model = LinearRNVP(input_dim=EMBEDDING_DIM, coupling_topology=[256, 256],
                      flow_n=12, batch_norm=True,
                      mask_type='odds', conditioning_size=2,
                      use_permutation=True, single_function=True).to(device)

nf_optimizer = torch.optim.Adam(nf_model.parameters(), lr=1e-4, weight_decay=1e-5)

NF_EPOCHS = 25

### Training the Conditional Normalizing Flow
We will now train the RealNVP model to learn the distribution of the normalized pet image embeddings, conditioned on whether the image is a cat or a dog.

1️⃣ **Forward Pass** → Transform a pet embedding `emb` and its one-hot encoded label `y` into a latent vector `u`.

2️⃣ **Compute Loss** → Maximize the log-likelihood of this transformation.

3️⃣ **Backward Pass** → Update the flow's parameters to better model the two conditional distributions.

In [ ]:
for j in range(NF_EPOCHS):
    nf_model.train()
    losses = []

    for batch_idx, data in enumerate(tqdm(embedded_train_loader)):
        emb, y = data
        emb = emb.to(device)
        y = y.to(device)

        y = nn.functional.one_hot(y, 2).to(device).float()

        u, log_det = nf_model.forward(emb, y=y)
        prior_logprob = nf_model.logprob(u)
        log_prob = -torch.mean(prior_logprob.sum(1) + log_det)

        losses.append(log_prob.item())

        nf_model.zero_grad()
        log_prob.backward()
        nf_optimizer.step()

    print(f'Epoch #{j + 1}, Loss: {sum(losses) / len(losses):.4f}')


### Evaluating the Model

In [ ]:
sample_n = 8
f, axs = plt.subplots(nrows=2, ncols=sample_n, figsize=(16, 4))

for ax in axs:
    for a in ax:
        a.set_xticklabels([])
        a.set_yticklabels([])
        a.set_aspect('equal')

f.subplots_adjust(wspace=0, hspace=0)

nf_model.eval()
autoencoder.eval()
with torch.no_grad():
    species_names = ['Cat', 'Dog']

    for i in range(2):
        y = torch.nn.functional.one_hot(torch.tensor([i] * sample_n), 2).to(device).float()
        emb, d = nf_model.sample(sample_n, y=y, return_logdet=True)

        # Denormalize embeddings before passing to decoder
        emb_denorm = emb * emb_std.to(device) + emb_mean.to(device)

        z = autoencoder.decode(emb_denorm)

        d_sorted = d.sort(0)[1].flip(0)
        z = z[d_sorted]
        z = (z + 1) / 2
        z = torch.clamp(z, 0, 1).cpu()

        for j in range(sample_n):
            img = z[j].permute(1, 2, 0)
            axs[i][j].imshow(img)
            if j == 0:
                axs[i][j].set_ylabel(species_names[i], rotation=0, labelpad=20)

plt.suptitle('Generated Pet Images: Cats (top) and Dogs (bottom)')
plt.show()

## **🔹 Exercise: Exploring the Generative Landscape**

The final image quality is a result of two models working in tandem. Tweaking the hyperparameters of either the Autoencoder or the Normalizing Flow can lead to better results(hopefully).

### **📝 Tasks**

1.  **Autoencoder Quality**: The Autoencoder was trained for 100 epochs. Try training it for longer (e.g., `AE_EPOCHS = 200`). Does a lower reconstruction loss in the AE lead to sharper, more realistic generations from the complete system?
2.  **Embedding Dimension**: Change `EMBEDDING_DIM` to `128` (more compression) or `512` (less compression). How does this trade-off affect the detail (e.g., fur texture) and variety of the generated pets?
3.  **Flow Complexity**: Adjust the `flow_n` (e.g., to `8` or `16`) and the `coupling_topology` in the `LinearRNVP` (e.g. `[512, 512]`). How does the flow's capacity impact its ability to model the subtle differences between cat and dog breeds?

Note:
-   A high-quality **Autoencoder is crucial**. Garbage in, garbage out; if the embeddings are poor, the Normalizing Flow cannot generate good images.
    - This was a very big issue when I was creating the notebook 🙃

### Contributed by: Ali Habibullah.